In [ ]:
# not autoreloading - raises TypeError in super().__init__() calls for models
from utils import *

from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
from time import time

In [ ]:
glove = setup_glove()
print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]
with open('../data/data.json', 'r') as f:
    data = json.load(f)

In [ ]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("ID:", example_X[0])
print("Context:", example_X[1])
print("Question:", example_X[2])
print("Answer Span:", example_y)
print("Answer:", example_X[3])
X, y = make_data([example_X], [example_y])
pprint(y)
print(get_answer_span(y[0], example_X[1]))
num_ex = 500
X_pass, y_pass = make_data(data['X_train'][:num_ex], data['y_train'][:num_ex])

In [ ]:
from models import *

conf = {"vocab": glove.vectors,
    "learning_rate": 0.1,
        "epochs": 5,
        "hidden_size": 32,
       "batch_size": 50,
       "opt": "Adamax",
        "n_layers": 1}
model = ModelV2(conf)
print(model, model.lr, model.hidden_size, model.batch_size, model.opt)
model_name = "BiGRU_%s_%s_%s_%s"%(num_ex, model.epochs, model.hidden_size, model.n_layers)
print(model_name)

In [ ]:
tic = time()
res = model.fit(X_pass, y_pass)
toc = time()
print("took", toc-tic, "seconds")
torch.save(model, '../evaluation/models/%s'%model_name)

In [ ]:
model = torch.load('../evaluation/models/%s'%model_name)
print(model)
print(model_name)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(range(len(res))), res)
plt.show()

In [ ]:
num_test = 100
X_test_data, y_test_data = make_data(data['X_test'][:num_test], data['y_test'][:num_test])

In [ ]:
dev_results = {}
seen_contexts = set()
seen_questions = set()
from hashlib import sha1

for i, (x, y) in enumerate(zip(X_test_data[:num_test], y_test_data[:num_test])):
    qid = data['X_test'][i][0]
    if qid not in seen_questions:
        seen_questions.add(qid)
    else:
        continue
    c = data['X_test'][i][1]
    context_h = sha1(c.encode('utf-8')).hexdigest()
    if context_h not in seen_contexts:
        print(c)
        print("-"*30)
        seen_contexts.add(context_h)
    q = data['X_test'][i][2]
    print("QUESTION: ", q)
    a = data['X_test'][i][3]
    print("ANSWER: ", a)
    res = model.predict([x], bs=1).data.tolist()[0]
    if res[0]>res[1]:
        res[0], res[1] = res[1], res[0]
    ans = get_answer_span(res, c)
    print("PRED_SPAN: ", res)
    print("PREDICTED: ", ans)
    dev_results[qid] = ans
    print("="*50)
with open('../data/run_%sdata_%sepochs.json'%(num_ex, model.epochs), 'w') as f:
    json.dump(dev_results, f)

In [ ]:
x = torch.ones((1, 25, 32))
x.repeat(600, 1, 1).size()